**EXTRAIR OS DADOS**

Importar bibliotecas necessárias

In [94]:
import pandas as pd # Para manipulação de databeses e dataframes

Obter os caminhos dos arquivos csv

In [95]:
path_cargos = 'data_raw/cargos.csv'
path_estruturas = 'data_raw/estruturas.csv'
path_funcionarios = 'data_raw/funcionarios.csv'

Ler arquivos CSV como DataFrames

In [96]:
# Usar encoding:'utf-8' para garantir que os dados sigam os padrões de acentuação brasileiro.

df_cargos = pd.read_csv(path_cargos, encoding='utf-8')
df_estruturas = pd.read_csv(path_estruturas, encoding='utf-8')
df_funcionarios = pd.read_csv(path_funcionarios, encoding='utf-8')

In [97]:
# Visualiza os dados extraídos
df_funcionarios

,id_funcionario,id_cargo,id_estrutura,estado,data_admissao,data_desligamento
0,1,16,6,São Paulo,2026-01-12,NaN
1,2,11,10,Santa Catarina,2025-05-22,NaN
2,3,1,3,São Paulo,2024-07-05,NaN
3,4,1,8,São Paulo,2024-12-01,NaN
4,5,2,5,Paraná,2024-12-13,NaN
...,...,...,...,...,...,...
95,96,12,2,São Paulo,2024-12-30,NaN
96,97,14,4,Santa Catarina,2025-04-20,NaN
97,98,7,9,São Paulo,2024-09-28,NaN
98,99,12,1,São Paulo,2025-04-19,2025-11-30


**TRANSFORMAR OS DADOS**

Transformar as colunas de data para o tipo 'datetime'

In [98]:
# Cria uma lista com todos os dataframes
df_list = [df_cargos, df_estruturas, df_funcionarios]

In [99]:
# Cria uma lista com todos os nomes das colunas de data
date_columns = ['start_date', 'end_date', 'data_admissao', 'data_desligamento']

In [100]:
# Verifica o tipo atual das colunas de data
type(df_cargos['start_date'][0])

str

In [101]:
# Transforma o tipo de todas as colunas de data em todos os dataframes para o formato 'datetime'
for df in df_list:

    # Obter todas os nomes das colunas do dataframe atual
    df_columns = df.columns.tolist()

    for column in df_columns:
        # Verifica se o nome da atual coluna possui na lista de 'date_columns'
        if column in date_columns:
            # Transforma a coluna atual para o tipo 'datetime'
            df[column] = pd.to_datetime(df[column], errors='coerce')


In [102]:
# Verifica o novo tipo das colunas de data
df_cargos['start_date'][0]

Timestamp('2024-01-01 00:00:00')

**Unir as tabelas "funcionarios" e "cargos"**

Validação explícita para verificar se todos os id_cargo em df_funcionarios existem no df_cargos

In [103]:
validation_col = 'cargo_valido' # Coluna de validação
col_to_verify = 'id_cargo' # Coluna a ser validada

# Cria uma coluna que verifica se todos os id_cargo em df_funcionarios existem no df_cargos
df_funcionarios[validation_col] = df_funcionarios[col_to_verify].isin(df_cargos[col_to_verify])

# Verifica se possui algum valor False na coluna de validação
has_false = (df_funcionarios[validation_col] == False).any()

# Exclui a coluna de validação
df_funcionarios.drop(axis=1, columns=validation_col)

if has_false:
    print("A tabela funcionarios contém pelo menos um " + col_to_verify + " inválido.")
else:
    print("A tabela funcionarios não possui nenhum " + col_to_verify + " inválido.")



A tabela funcionarios não possui nenhum id_cargo inválido.


Unir as tabelas

In [104]:
df_merged = pd.merge(df_funcionarios, df_cargos, on='id_cargo', how='left') # LEFT JOIN para manter "funcionarios", mesmo que não exista um "cargo" correspondente

Filtrar "data_admissao" e "data_desligamento" 
para estar entre "start_date" e "end_date"

In [105]:
# Obtém todos NaT de "data_desligamento" e substitui por "end_date" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df_merged.loc[df_merged['data_desligamento'].isna(), 'data_desligamento'] = df_merged['end_date']


In [106]:
# Filtra o novo dataframe unido
df_filtered = df_merged[
    (df_merged['data_admissao'] >= df_merged['start_date']) &
    (df_merged['data_desligamento'] <= df_merged['end_date'])
]

In [107]:
# Valida o número de linhas do antigo e o novo dataframe filtrado
print("Total de linhas de df_merged: " + str(df_merged.__len__()))
print("Total de linhas de df_filtered: " + str(df_filtered.__len__()))

Total de linhas de df_merged: 100
Total de linhas de df_filtered: 93


Visualizar a nova tabela unida e filtrada

In [108]:
# Confirma a nova tabela filtrada (funcionarios e cargos)
df_filtered

,id_funcionario,id_cargo,id_estrutura,estado,data_admissao,data_desligamento,cargo_valido,nome_cargo,start_date,end_date
0,1,16,6,São Paulo,2026-01-12,2026-03-31,True,Gerente de Produto,2025-03-01,2026-03-31
1,2,11,10,Santa Catarina,2025-05-22,2025-10-31,True,Consultor de TI,2024-10-01,2025-10-31
2,3,1,3,São Paulo,2024-07-05,2024-12-31,True,Analista de Banco de Dados,2024-01-01,2024-12-31
3,4,1,8,São Paulo,2024-12-01,2024-12-31,True,Analista de Banco de Dados,2024-01-01,2024-12-31
4,5,2,5,Paraná,2024-12-13,2025-01-31,True,Analista de Suporte,2024-01-01,2025-01-31
...,...,...,...,...,...,...,...,...,...,...
95,96,12,2,São Paulo,2024-12-30,2025-11-30,True,Desenvolvedor Front-end,2024-11-01,2025-11-30
96,97,14,4,Santa Catarina,2025-04-20,2026-01-31,True,Engenheiro de Dados,2025-01-01,2026-01-31
97,98,7,9,São Paulo,2024-09-28,2025-06-30,True,Administrador de Sistemas,2024-06-01,2025-06-30
98,99,12,1,São Paulo,2025-04-19,2025-11-30,True,Desenvolvedor Front-end,2024-11-01,2025-11-30


**Unir as tabelas anteriores e a tabela "estruturas"**

Validação explícita para verificar se todos os id_estruturas em df_funcionarios existem no df_estruturas

In [109]:
validation_col = 'estrutura_valido' # Coluna de validação
col_to_verify = 'id_estrutura' # Coluna a ser validada

# Cria uma coluna que verifica se todos os id_estrutura em df_funcionarios existem no df_estruturas
df_funcionarios[validation_col] = df_funcionarios[col_to_verify].isin(df_estruturas[col_to_verify])

# Verifica se possui algum valor False na coluna de validação
has_false = (df_funcionarios[validation_col] == False).any()

# Exclui a coluna de validação
df_funcionarios.drop(axis=1, columns=validation_col)

if has_false:
    print("A tabela funcionarios contém pelo menos um " + col_to_verify + " inválido.")
else:
    print("A tabela funcionarios não possui nenhum " + col_to_verify + " inválido.")

A tabela funcionarios não possui nenhum id_estrutura inválido.


Unir as tabelas

In [110]:
df_merged_all = pd.merge(df_filtered, df_estruturas, on='id_estrutura', how='left') # LEFT JOIN para manter os dados anteriores

Renomeia a colunas start e end para uma melhor manipulação posterior

In [111]:
df_merged_all = df_merged_all.rename(columns={
    'start_date_x': 'start_date_cargo',
    'end_date_x': 'end_date_cargo',
    'start_date_y': 'start_date_estrutura',
    'end_date_y': 'end_date_estrutura'
})

Filtra "data_admissao" e "data_desligamento" para estar entre "start_date_estrutura" e "end_date_estrutura"

In [112]:
# Filtrar o dataframe com todas as tabelas unidas
df_filtered_all = df_merged_all[
    (df_merged_all['data_admissao'] >= df_merged_all['start_date_estrutura']) &
    ((df_merged_all['data_desligamento'] <= df_merged_all['end_date_estrutura']))
]

In [113]:
# Valida o tamanho de linhas da nova tabela após o filtro de tempo
print("Tamanho da nova tabela filtrada: " + str(df_filtered_all.__len__()))

Tamanho da nova tabela filtrada: 26


Remove as colunas "start_date" e "end_date"

In [114]:
# Remove colunas
df_formated = df_filtered_all.drop(axis=1, columns=[
    'start_date_cargo', 'end_date_cargo', 'start_date_estrutura', 'end_date_estrutura'
])

Reorganizar as colunas

In [115]:
# Reoganiza as colunas na ordem desejada (Para melhor visualização do usuário final)
df_formated = df_formated[['id_funcionario', 'data_admissao', 'data_desligamento', 'estado', 'id_cargo', 'nome_cargo', 'id_estrutura', 'nome_estrutura']]

Visualizar a Tabela Final formatada

In [116]:
# Obter a tabela final com todas as tabelas unidas e filtradas
df_formated

,id_funcionario,data_admissao,data_desligamento,estado,id_cargo,nome_cargo,id_estrutura,nome_estrutura
2,3,2024-07-05,2024-12-31,São Paulo,1,Analista de Banco de Dados,3,GER TECNOLOGIA DA INFORMAÇÃO
3,4,2024-12-01,2024-12-31,São Paulo,1,Analista de Banco de Dados,8,GER VENDAS
4,5,2024-12-13,2025-01-31,Paraná,2,Analista de Suporte,5,GER FINANCEIRO
5,6,2024-10-22,2025-02-28,Santa Catarina,3,Analista de Segurança da Informação SR,7,GER OPERAÇÕES
11,13,2024-10-12,2025-05-31,São Paulo,6,Engenheiro de Software,10,GER DESENVOLVIMENTO DE NEGÓCIOS
12,14,2024-05-15,2025-01-31,São Paulo,2,Analista de Suporte,6,GER MARKETING
19,21,2025-01-24,2025-08-31,São Paulo,9,Gerente de TI,10,GER DESENVOLVIMENTO DE NEGÓCIOS
24,27,2024-11-07,2024-12-31,Paraná,1,Analista de Banco de Dados,2,GER INTELIGENCIA COMERCIAL B2B
28,32,2025-01-04,2025-01-31,Rio de Janeiro,2,Analista de Suporte,7,GER OPERAÇÕES
33,37,2024-12-19,2024-12-31,São Paulo,1,Analista de Banco de Dados,8,GER VENDAS


**CARREGAR OS DADOS**

Carrega a tabela formatada para o formato csv no caminho de destino

In [117]:
end_path = 'data_processed/'
file_name = 'tabela_formatada.csv'

df_formated.to_csv(end_path + file_name, index=False, encoding='utf-8')

**OBTER INDICADORES**

Preparação dos dados dos indicadores

In [118]:
# Cria duas novas colunas com o total de "data_admissao" and "data_desligamento" por mês
df_formated['mes_admissao'] = df_formated['data_admissao'].dt.to_period('M')
df_formated['mes_desligamento'] = df_formated['data_desligamento'].dt.to_period('M')

Obter o total de funcionários contratados por mês

In [119]:
# Obter a tabela (serie) com o total de admissão por mês
s_admitidos = df_formated['mes_admissao'].value_counts().sort_index()
s_admitidos = s_admitidos.rename('admitidos') # Da um nome a tabela (serie)
s_admitidos

mes_admissao
2024-03    1
2024-05    1
2024-06    2
2024-07    3
2024-08    2
2024-09    3
2024-10    2
2024-11    1
2024-12    4
2025-01    3
2025-02    1
2025-04    1
2025-05    2
Freq: M, Name: admitidos, dtype: int64

Obter o total de funcionários desligados por mês

In [120]:
# Obter a tabela (serie) com o total de desligamento por mês
s_desligados = df_formated['mes_desligamento'].value_counts().sort_index()
s_desligados = s_desligados.rename('desligados') # Da um nome a tabela (serie)
s_desligados

mes_desligamento
2024-12    7
2025-01    5
2025-02    2
2025-03    5
2025-04    1
2025-05    1
2025-06    2
2025-07    1
2025-08    2
Freq: M, Name: desligados, dtype: int64

Obter o total de funcionários ativos por mês

In [121]:
# Obter o primeiro e o último mês registrado na tabela
first_date = df_formated['mes_admissao'].min()
last_date = df_formated['mes_desligamento'].max()

print('Data inicial: ' + str(first_date))
print('Data final: ' + str(last_date))

# Obter todos os meses entre "first_date" e "last_date"
all_months = pd.period_range(first_date, last_date, freq='M')

print("Total de meses entre as duas: " + str(all_months.__len__()))

Data inicial: 2024-03
Data final: 2025-08
Total de meses entre as duas: 18


In [122]:
# Obter a tabela (serie) com o total de funcionários ativos por mês
s_ativos = pd.Series(index=all_months, dtype='Int64', name='ativos')

for mes in all_months:
    s_ativos[mes] = df_formated[
        (df_formated['mes_admissao'] <= mes) &
        (df_formated['mes_desligamento'] >= mes)
    ].shape[0]

s_ativos = s_ativos.rename_axis('mes')
s_ativos

mes
2024-03     1
2024-04     1
2024-05     2
2024-06     4
2024-07     7
2024-08     9
2024-09    12
2024-10    14
2024-11    15
2024-12    19
2025-01    15
2025-02    11
2025-03     9
2025-04     5
2025-05     6
2025-06     5
2025-07     3
2025-08     2
Freq: M, Name: ativos, dtype: Int64

Criar um único dataframe com todos os indicadores

In [123]:
# Cria uma lista com todas as tabelas (series) de indicadores
series_list = [s_ativos, s_admitidos, s_desligados]

# Transforma as series em um dataframe (Unidos pelos meses)
df_indicadores = pd.concat(series_list, axis=1)

# Transforma todos os valores 'NaN' em 0
df_indicadores = df_indicadores.fillna(0).astype({
    col: 'Int64' for col in df_indicadores.select_dtypes(include='float').columns # Transforma todas as colunas do tipo 'float' no tipo 'int'
    })

# Transforma o index em uma coluna
i_column_name = 'mes'
df_indicadores = df_indicadores.reset_index(names=i_column_name)

Visualizar os indicadores

In [124]:
# Obter a tabela final de indicadores
df_indicadores

,mes,ativos,admitidos,desligados
0,2024-03,1,1,0
1,2024-04,1,0,0
2,2024-05,2,1,0
3,2024-06,4,2,0
4,2024-07,7,3,0
5,2024-08,9,2,0
6,2024-09,12,3,0
7,2024-10,14,2,0
8,2024-11,15,1,0
9,2024-12,19,4,7


**CARREGAR TABELA INDICADORES**

Carrega a tabela de indicadores formatada para o formato csv no caminho de destino

In [125]:
end_path = 'data_processed/'
file_name = 'indicadores.csv'

df_indicadores.to_csv(end_path + file_name, index=False, encoding='utf-8')